# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [87]:
#Libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [69]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [70]:
spaceship.drop(columns = ["PassengerId", "Name"], inplace=True)
spaceship["Cabin"] = spaceship["Cabin"].str[:1]

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [71]:
num = spaceship.select_dtypes("number")
cat = spaceship.select_dtypes("object")

In [72]:
normalizer = MinMaxScaler()
normalizer.fit(num)
num_norm = normalizer.transform(num)
num_norm = pd.DataFrame(num_norm, columns = num.columns)

In [73]:
cat = pd.get_dummies(cat)

In [80]:
df = num.join(cat).join(spaceship["Transported"])
df.dropna(inplace=True)

**Perform Train Test Split**

In [81]:
features = df.drop(columns = ["Transported"])
target = df["Transported"]

In [82]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [83]:
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000)

In [84]:
bagging_reg.fit(X_train, y_train)

BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=20),
                 max_samples=1000, n_estimators=100)

In [89]:
pred = bagging_reg.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", bagging_reg.score(X_test, y_test))

MAE 0.2760882481880415
RMSE 0.3736327670393942
R2 score 0.4414316468304461


- Random Forests

In [90]:
forest = RandomForestRegressor(n_estimators=100,
                             max_depth=20)

In [92]:
forest.fit(X_train, y_train)

RandomForestRegressor(max_depth=20)

In [93]:
pred = forest.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", forest.score(X_test, y_test))

MAE 0.2643012451013784
RMSE 0.38041066497112813
R2 score 0.4209823772719835


- Gradient Boosting

In [94]:
gb_reg = GradientBoostingRegressor(max_depth=20,
                                   n_estimators=100)

In [95]:
gb_reg.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=20)

In [96]:
pred = gb_reg.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", gb_reg.score(X_test, y_test))

MAE 0.26476995735044834
RMSE 0.4452685376810517
R2 score 0.20671282630467425


- Adaptive Boosting

In [98]:
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=20),
                            n_estimators=100)

In [99]:
ada_reg.fit(X_train, y_train)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=20),
                  n_estimators=100)

In [100]:
pred = ada_reg.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", ada_reg.score(X_test, y_test))

MAE 0.23986316397088514
RMSE 0.4229818818618248
R2 score 0.2841369541332881


Which model is the best and why?

Among the 4 models, Bagging and Pasting seems to be the best model since it has lowest RMSE=0.3736 and highest R2=0.4414